In [ ]:
!pip install --upgrade plotly

In [ ]:
from fbprophet.plot import plot_plotly

In [ ]:
import os
import pandas as pd
import numpy as np
import scipy
from fbprophet import Prophet
from fbprophet.plot import plot_plotly

from google.colab import drive
drive.mount('/content/gdrive')

os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Covid19Hackathon"

from sklearn.metrics import mean_squared_error
from math import sqrt
# pd.options.plotting.backend = "plotly"

pd.set_option('display.max_rows', 750)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)
pd.set_option('display.max_colwidth', 200)

from IPython.display import display

%cd /content/gdrive/My Drive/Covid19Hackathon

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Covid19Hackathon


In [ ]:
def state_data_vivek(df):
  state_wise_daily = df
  base_data_df = pd.DataFrame()
  base_data_df['Date'] = state_wise_daily.Date_time.unique()
  base_data_df['Key'] = 1
  # base_data_df['Year'] = base_data_df['Date'].apply(lambda x: x.strftime('%d-%m-%Y'))
  print(base_data_df.head(5))
  print(base_data_df.shape)
  list_states = state_wise_daily.columns.tolist()
  rm_ilst = ['Date', 'Status', 'TT', 'Date_time']
  for i in rm_ilst:
      list_states.remove(i)#,"Status","Date_time","TT"])
  state_df = pd.DataFrame()
  state_df["State_cd"] = list_states
  state_df["Key"] = 1

  base_data_df = base_data_df.merge(state_df, on='Key', how='left')
  base_data_df = base_data_df.drop(columns = ['Key'])
  print(base_data_df.shape)
  base_data_df.head(2)

  conf_list = []
  dead_lsit = []
  recv_list = []
  for i in range(0, len(base_data_df['Date'])):
      val = 0
      val = state_wise_daily[(state_wise_daily['Date_time'] == base_data_df['Date'][i]) & (state_wise_daily['Status'] == 'Confirmed')][base_data_df['State_cd'][i]].values[0]
      conf_list.append(val)
  for i in range(0, len(base_data_df['Date'])):
      val = 0
      val = state_wise_daily[(state_wise_daily['Date_time'] == base_data_df['Date'][i]) & (state_wise_daily['Status'] == 'Deceased')][base_data_df['State_cd'][i]].values[0]
      dead_lsit.append(val)
  for i in range(0, len(base_data_df['Date'])):
      val = 0
      val = state_wise_daily[(state_wise_daily['Date_time'] == base_data_df['Date'][i]) & (state_wise_daily['Status'] == 'Recovered')][base_data_df['State_cd'][i]].values[0]
      recv_list.append(val)
  base_data_df['Confirmed'] = conf_list
  base_data_df['Deceased']  = dead_lsit
  base_data_df['Recovered'] = recv_list
  return(base_data_df)


# Module for creating time-series variables

def ts_features(data_, n, widths = [7,14,21]): # ndays behind 21 days ## data contains DatexState Cases # State = [] means by default it predicts for Overall

  data = data_.copy() 
  data.set_index(['Date'], inplace = True)  

  merged = data.copy()
  for i in range(1,n):
    df =  data.copy()
    df = df.shift(i)      # shift with lag step=1
    df.columns = [j.replace(str(1),str(i+1)) for j in df.columns]
    merged = merged.merge(df, how = 'left', left_index=True, right_index=True)

  for width in widths:
    df =  data.copy()
    df = df.rolling(width).mean()      # pull out the State, shift with lag step=1, roll
    df.columns = [j.replace("_lag1", "") + "_rolling_last" + str(width) for j in df.columns]
    merged = merged.merge(df, how = 'left', left_index=True, right_index=True)

  merged = merged.reset_index()
  merged['dayofweek'] = merged['Date'].dt.dayofweek
  merged['isweekend'] = np.where(merged['dayofweek'].isin([5,6]), 1, 0)

  return(merged)

In [ ]:
indv_data_list = [pd.read_csv("https://api.covid19india.org/csv/latest/raw_data" + str(i+1) + ".csv") for i in range(12)]
indv_data = pd.concat(indv_data_list, axis = 0).reset_index(drop = True)
state_code_mapping = (indv_data['State code'] + "-" + indv_data['Detected State']).dropna(axis = 0).unique()
state_code_mapping = {i.split("-")[0]:i.split("-")[1] for i in list(state_code_mapping)}

_state_daily = pd.read_csv("https://api.covid19india.org/csv/latest/state_wise_daily.csv")
_state_daily['Date_time'] = pd.to_datetime(_state_daily['Date'] + "20", format = "%d-%b-%Y")
_state_daily.rename(state_code_mapping, axis = 1, inplace = True)
_state_daily.drop('DD', axis = 1, inplace = True)

state_daily = state_data_vivek(_state_daily).rename({'State_cd': 'State'}, axis = 1)


# Adding Testing Data (to be removed later/ cummax to be taken in model/logistic growth)
state_test_daily = pd.read_csv('https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv')
print(state_test_daily.shape)
# Select & Rename
state_test_daily = state_test_daily[['Updated On', 'State', 'Total Tested', 'Positive']].rename(columns={'Updated On' : 'Date', 'State' : 'State', 'Total Tested' : 'Tests', 'Positive' : 'Cases' })
# state_test_daily = state_test_daily[['Updated On', 'State', 'Test positivity rate']].rename(columns={'Updated On' : 'Date', 'State' : 'State', 'Test positivity rate': 'Positivity Rate' })
print(state_test_daily.shape)

# Add Date Mapping
state_test_daily['Date'] = pd.to_datetime(state_test_daily['Date'], format = '%d/%m/%Y')
state_test_daily = state_test_daily.drop_duplicates()
print(state_test_daily.shape)
state_test_daily.head(2)

# base_data = pd.read_csv(os.getcwd() + "/XgBoost/XgbData.csv")
# base_data['Date'] = pd.to_datetime(base_data['Date'],format = "%Y-%m-%d")
# base_data = base_data[['Date', 'State', 'Confirmed']]

base_data = state_daily[['Date', 'State', 'Confirmed']]
base_data = base_data.merge(state_test_daily, how = 'left', on = ['Date', 'State'])

display(base_data.head())

        Date  Key
0 2020-03-14    1
1 2020-03-15    1
2 2020-03-16    1
3 2020-03-17    1
4 2020-03-18    1
(146, 2)
(5402, 2)
(3967, 28)
(3967, 4)
(3966, 4)


,Date,State,Confirmed,Tests,Cases
0,2020-03-14,Andaman and Nicobar Islands,0,NaN,NaN
1,2020-03-14,Andhra Pradesh,1,NaN,NaN
2,2020-03-14,Arunachal Pradesh,0,NaN,NaN
3,2020-03-14,Assam,0,NaN,NaN
4,2020-03-14,Bihar,0,NaN,NaN


In [ ]:
st_list = ['Maharashtra', 'Tamil Nadu', 'Delhi', 'Karnataka', 'Andhra Pradesh', 'Uttar Pradesh', 'West Bengal', 'Gujarat', 'Telangana', 'Rajasthan', 'Bihar', 'Assam', 'Haryana', 'Madhya Pradesh', 'Odisha', 'Kerala', 'Jammu and Kashmir', 'Punjab', 'Jharkhand', 'Chhattisgarh', 'Uttarakhand', 'Goa', 'Tripura', 'Puducherry', 'Manipur', 'Himachal Pradesh', 'Ladakh', 'Nagaland', 'Arunachal Pradesh', 'Dadra and Nagar Haveli and Daman and Diu', 'Chandigarh', 'Meghalaya', 'Sikkim', 'Mizoram', 'Andaman and Nicobar Islands']
cnf_list = [10483, 5880, 1192, 6670, 10171, 4404, 2954, 1034, 2207, 422, 3646, 2372, 751, 734, 1833, 1251, 473, 1035, 708, 523, 369, 333, 128, 241, 124, 28, 3, 77, 93, 44, 57, 51, 29, 21, 96]
Aug7 = pd.DataFrame(zip(st_list, cnf_list), columns = ['State', 'Confirmed'])
Aug7['Date'] = pd.Timestamp('2020-08-07')
base_data = base_data.append(Aug7)

In [ ]:
def metrics(state, y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    perf = pd.DataFrame({'State': [state], 'RMSE': [rmse], 'MAPE': [mape]})
    print(f"{state}\t{sum(y_true)}\t{rmse}\t{mape}")
    return(perf)

def model_run_forecast(data, growth_type, start_date, end_date,n_days, state): #Prophet Model

  model_data = data[(data['Date'] >= start_date) & (data['Date'] <= end_date)]

  prophet = Prophet(growth=growth_type, daily_seasonality= False, yearly_seasonality= False, seasonality_mode = 'multiplicative')# ,changepoints = [pd.Timestamp("2020-06-20 00:00:00")])
  model_data = model_data[['Date', 'Confirmed']].rename({'Date':'ds', 'Confirmed': 'y'}, axis = 1)

  if growth_type == 'logistic':
    model_data['floor'] = 0
    model_data['cap'] = 1.5*model_data['y'].max()
  prophet.fit(model_data)
  
  future = prophet.make_future_dataframe(periods=n_days, freq='d')
  if growth_type == 'logistic':
    future['floor'] = 0
    future['cap'] = 1.5*model_data['y'].max()
  forecast = prophet.predict(future)
  # display(plot_plotly(prophet, forecast))
  
  act_preds = forecast[['ds', 'yhat']].rename({'ds':'Date', 'yhat':'Predictions'}, axis = 1)
  act_preds = act_preds.merge(data[['Date', 'Confirmed']], how = 'left', on = 'Date')
  act_preds = act_preds[(act_preds['Date'] > end_date)].reset_index()
  act_preds['Predictions'] = act_preds['Predictions'].astype(int)
  # display(act_preds)
  # display(act_preds.set_index('Date').plot())
  
  perf = pd.DataFrame()#metrics(state, act_preds['Confirmed'], act_preds['Predictions'])
  perf['Date'] = start_date
  return(act_preds, perf)

In [ ]:
base_data.tail()

,Date,State,Confirmed,Tests,Cases
30,2020-08-07,Chandigarh,57,NaN,NaN
31,2020-08-07,Meghalaya,51,NaN,NaN
32,2020-08-07,Sikkim,29,NaN,NaN
33,2020-08-07,Mizoram,21,NaN,NaN
34,2020-08-07,Andaman and Nicobar Islands,96,NaN,NaN


In [ ]:
states = ["Maharashtra", "Tamil Nadu", "Delhi", "Karnataka", "Andhra Pradesh", "Uttar Pradesh", "West Bengal", "Gujarat", "Telangana", "Rajasthan", "Bihar", "Assam", "Haryana", "Madhya Pradesh", "Odisha", "Kerala", "Jammu and Kashmir", "Punjab", "Jharkhand", "Chhattisgarh", "Uttarakhand", "Goa", "Tripura", "Puducherry", "Manipur", "Himachal Pradesh", "Ladakh", "Nagaland", "Arunachal Pradesh", "Dadra and Nagar Haveli and Daman and Diu", "Chandigarh", "Meghalaya", "Sikkim", "Mizoram", "Andaman and Nicobar Islands"]

group_1 = ["Karnataka", "Telangana", "Andhra Pradesh", "Dadra and Nagar Haveli and Daman and Diu", "Puducherry", "Assam", "Odisha", "Goa", "Arunachal Pradesh", "Kerala"]
group_2 = ["Nagaland", "Sikkim", "Tamil Nadu", "West Bengal", "Uttar Pradesh", "Manipur", "Bihar", "Haryana", "Jharkhand", "Mizoram", "Chhattisgarh", "Punjab", "Maharashtra", "Delhi", "Tripura", "Jammu and Kashmir"]
group_3 = ["Himachal Pradesh", "Ladakh", "Rajasthan", "Uttarakhand"]
group_4 = ["Gujarat", "Madhya Pradesh", "Meghalaya", "Chandigarh"]

last_few_states = ["Gujarat", "Telangana", "Rajasthan", "Bihar", "Assam", "Haryana", "Madhya Pradesh", "Odisha", "Jammu and Kashmir", "Jharkhand", "Manipur", "Nagaland", "Arunachal Pradesh", "Dadra and Nagar Haveli and Daman and Diu", "Meghalaya", "Andaman and Nicobar Islands"]
final_multistep_states = ["Tamil Nadu", "Delhi", "Karnataka", "Gujarat", "Telangana", "Rajasthan", "Bihar", "Assam", "Haryana", "Madhya Pradesh", "Odisha", "Jammu and Kashmir" , "Punjab", "Jharkhand", "Chhattisgarh", "Manipur", "Nagaland", "Arunachal Pradesh" ,"Dadra and Nagar Haveli and Daman and Diu", "Meghalaya", "Andaman and Nicobar Islands"]

In [ ]:
# Delhi change_points June 15, 20, 23  
# base_data[base_data['State'] == 'Delhi'].set_index('Date')[['Confirmed']].plot()

In [ ]:
(pd.Timestamp("6-5-2020 00:00:00") - pd.Timestamp("6-4-2020 00:00:00")).days

1

In [ ]:
final_multistep_selected_dates = {"Tamil Nadu": pd.Timestamp("6-5-2020 00:00:00"),
"Delhi": pd.Timestamp("6-26-2020 00:00:00"),
"Karnataka": pd.Timestamp("6-26-2020 00:00:00"),
"Gujarat": pd.Timestamp("6-26-2020 00:00:00"),
"Telangana": pd.Timestamp("7-3-2020 00:00:00"),
"Rajasthan": pd.Timestamp("6-26-2020 00:00:00"),
"Bihar": pd.Timestamp("6-26-2020 00:00:00"),
"Assam": pd.Timestamp("5-1-2020 00:00:00"),
"Haryana": pd.Timestamp("5-8-2020 00:00:00"),
"Madhya Pradesh": pd.Timestamp("6-26-2020 00:00:00"),
"Odisha": pd.Timestamp("6-26-2020 00:00:00"),
"Jammu and Kashmir":pd.Timestamp("6-16-2020 00:00:00"),
"Punjab": pd.Timestamp("7-3-2020 00:00:00"),
"Jharkhand": pd.Timestamp("7-3-2020 00:00:00"),
"Chhattisgarh": pd.Timestamp("7-3-2020 00:00:00"),
"Manipur": pd.Timestamp("6-12-2020 00:00:00"),
"Arunachal Pradesh": pd.Timestamp("6-26-2020 00:00:00"), 
"Nagaland": pd.Timestamp("6-26-2020 00:00:00"),
"Dadra and Nagar Haveli and Daman and Diu": pd.Timestamp("6-5-2020 00:00:00"),
"Meghalaya": pd.Timestamp("5-29-2020 00:00:00"),
"Andaman and Nicobar Islands": pd.Timestamp("6-5-2020 00:00:00")
}

last_date = pd.Timestamp("8-07-2020 00:00:00")

final_multistep_selected_dates = {"Tamil Nadu": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Tamil Nadu"]) ,
"Delhi": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Delhi"]) ,
"Karnataka": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Karnataka"]) ,
"Gujarat": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Gujarat"]) ,
"Telangana": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Telangana"]) ,
"Rajasthan": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Rajasthan"]) ,
"Bihar": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Bihar"]) ,
"Assam": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Assam"]) ,
"Haryana": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Haryana"]) ,
"Madhya Pradesh": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Madhya Pradesh"]) ,
"Odisha": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Odisha"]) ,
"Jammu and Kashmir": last_date -(pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Jammu and Kashmir"]) ,
"Punjab": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Punjab"]) ,
"Jharkhand": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Jharkhand"]) ,
"Chhattisgarh": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Chhattisgarh"]) ,
"Manipur": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Manipur"]) ,
"Arunachal Pradesh": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Arunachal Pradesh"]) , 
"Nagaland": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Nagaland"]) ,
"Dadra and Nagar Haveli and Daman and Diu": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Dadra and Nagar Haveli and Daman and Diu"]) ,
"Meghalaya": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Meghalaya"]) ,
"Andaman and Nicobar Islands": last_date - (pd.Timestamp("7-17-2020 00:00:00") - final_multistep_selected_dates["Andaman and Nicobar Islands"] )
}

In [ ]:
pred_df = []
conf_df = []
i = 1
perf_df = pd.DataFrame()

for state in final_multistep_states:#last_few_states:#states:

  for day in [0, 7, 14, 21, 28, 35, 42, 49, 56, 63][0:1]:
    start_date = final_multistep_selected_dates[state]# pd.Timestamp("2020-06-16 00:00:00") + pd.Timedelta(day, 'd')#start_dates_dict[state]#
    end_date = pd.Timestamp("2020-08-07 00:00:00")
  
    model_data = base_data[base_data['State'].isin([state])]#.groupby('Date').sum().reset_index()
    print(state)
    predictions, perf = model_run_forecast(model_data, 'linear' , start_date, end_date, 21, state)
    perf_df = perf_df.append(perf)

      # display(model_data.set_index('Date')['Confirmed'].plot())
    pred_df = pred_df + [predictions[['Predictions']].rename({'Predictions':state}, axis = 1)]
    conf_df = conf_df + [predictions[['Confirmed']].rename({'Confirmed':state}, axis = 1)]
    i = i + 1

Tamil Nadu


INFO:fbprophet:n_changepoints greater than number of observations. Using 16.


Delhi


INFO:fbprophet:n_changepoints greater than number of observations. Using 16.


Karnataka


...

Chhattisgarh
Manipur


INFO:fbprophet:n_changepoints greater than number of observations. Using 16.


Nagaland


INFO:fbprophet:n_changepoints greater than number of observations. Using 16.


Arunachal Pradesh
Dadra and Nagar Haveli and Daman and Diu
Meghalaya
Andaman and Nicobar Islands


In [ ]:
perf_df

,Date


In [ ]:
display(pd.concat(pred_df, axis = 1))
display(pd.concat(conf_df, axis = 1))

,Tamil Nadu,Delhi,Karnataka,Gujarat,Telangana,Rajasthan,Bihar,Assam,Haryana,Madhya Pradesh,Odisha,Jammu and Kashmir,Punjab,Jharkhand,Chhattisgarh,Manipur,Nagaland,Arunachal Pradesh,Dadra and Nagar Haveli and Daman and Diu,Meghalaya,Andaman and Nicobar Islands
0,6706,1048,6413,1012,2351,1183,4492,1826,865,789,1889,578,1228,924,307,116,193,72,53,60,73
1,6793,908,6465,991,1295,1268,3713,1624,810,940,1838,646,1091,734,313,78,147,52,56,46,80
2,6660,661,5938,934,1510,1265,3021,2308,809,803,1824,652,970,1129,297,77,237,70,42,38,71
3,6483,867,6732,944,1786,1240,3244,2693,785,788,1656,535,819,968,354,105,324,69,46,28,74
4,6568,924,6782,950,2353,1275,3416,2215,866,824,1695,608,1096,1046,313,113,132,86,46,23,93
5,6861,938,7678,948,2423,1245,3745,2462,854,821,2022,611,1211,906,402,110,128,77,47,44,82
6,6882,963,7192,928,2604,941,4437,2472,860,819,2306,577,1301,1138,453,141,157,94,57,61,90
7,7205,948,7180,909,2689,1273,5405,2109,921,815,2233,622,1566,1166,313,135,256,76,59,68,88
8,7293,820,7224,889,1477,1363,4446,1871,862,972,2164,695,1380,920,319,90,193,55,62,52,96
9,7145,596,6624,836,1718,1359,3602,2651,861,829,2140,701,1218,1404,303,88,309,74,47,43,84


,Tamil Nadu,Delhi,Karnataka,Gujarat,Telangana,Rajasthan,Bihar,Assam,Haryana,Madhya Pradesh,Odisha,Jammu and Kashmir,Punjab,Jharkhand,Chhattisgarh,Manipur,Nagaland,Arunachal Pradesh,Dadra and Nagar Haveli and Daman and Diu,Meghalaya,Andaman and Nicobar Islands
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
